In [1]:
from utils import *

## 1.hyperparameter

In [2]:
input_size = 1
hidden_size = 64
num_layers = 1
output_size = 1
seq_len = 10   # 预测序列长度
epoch = 500
learning_rate = 0.001

In [ ]:
if __name__ == "__main__":
    # checking if GPU is available
    device = torch.device("cpu")
    if (torch.cuda.is_available()):
        device = torch.device("cuda:0")
        print('Training on GPU.')
    else:
        print('No GPU available, training on CPU.')

    # 构建模型
    model = LSTMModel(input_size=input_size,
                      hidden_size=hidden_size,
                      output_size=output_size,
                      num_layers=num_layers)

    # dataset
    train_data, train_data_real = data_loader('B0005', seq_len, input_size)
    # issue3:sourceTensor.clone().detach().requires_grad_(True)
    train_data = torch.tensor(train_data, dtype=torch.float32)
    train_data_real = torch.tensor(train_data_real, dtype=torch.float32)
    #vailidation
    validation_data, validation_data_real = data_loader('B0006', seq_len, input_size)
    validation_data = torch.tensor(validation_data, dtype=torch.float32)
    validation_data_real = torch.tensor(validation_data_real, dtype=torch.float32)

    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    # criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 储存路径
    work_dir = './LSTM'
    # 添加tensorboard
    writer = SummaryWriter("{}/logs".format(work_dir))

    # device = torch.device("cuda:0")
    # model = model.to(device)
    # criterion = criterion.to(device)

    # 训练模型
    for epoch in range(epoch):
        print("-------epoch  {} -------".format(epoch + 1))
        # 训练步骤
        model.train()

        output = model(train_data)
        loss = criterion(output, torch.tensor(train_data_real, dtype=torch.float32).view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("train set Loss: {}".format(loss.item()))

        # 测试步骤
        model.eval()
        total_test_loss = 0
        total_accuracy = 0
        with torch.no_grad():  #用于在推断或验证阶段，当不需要计算梯度时，以提高效率和减少内存占用
            test_outputs = model(validation_data)
            loss = criterion(test_outputs, torch.tensor(validation_data_real, dtype=torch.float32).view(-1, 1))
            # total_test_loss = total_test_loss + loss.item()
            # accuracy = (outputs.argmax(1) == targets).sum()
            # total_accuracy = total_accuracy + accuracy
        print("test set Loss: {}".format(loss.item()))
    # Save the trained model
    torch.save(model.state_dict(), 'lstm_model.pth')